# Resume analysis with Azure Open AI

In [1]:
import os
import openai
import sys

from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

In [3]:
openai.api_version

'2023-05-15'

In [4]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

In [5]:
model = "gpt-35-turbo-16k"

## Testing

In [6]:
def QnA(prompt):
    """
    QnA with Azure Open AI
    """
    response = openai.ChatCompletion.create(
    engine=model,
    messages=[
        {"role": "user", "content": prompt},
    ],
    temperature=0.8,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    )
    print(response)

In [7]:
QnA("Who are you?")

{
  "id": "chatcmpl-7w6hvYjG1kFVtCLv1KlHNSG8KZeJO",
  "object": "chat.completion",
  "created": 1694083163,
  "model": "gpt-35-turbo-16k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "I am an AI language model developed by OpenAI. I am programmed to assist and engage in conversation with users. How may I assist you today?"
      }
    }
  ],
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 11,
    "total_tokens": 41
  }
}


In [8]:
QnA("What is Azure Open AI?")

{
  "id": "chatcmpl-7w6hxSn20v6NdBjZaM0mrWYPiA5z2",
  "object": "chat.completion",
  "created": 1694083165,
  "model": "gpt-35-turbo-16k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Azure Open AI is a cloud-based service provided by Microsoft Azure that allows developers to access and use OpenAI's GPT-3 (Generative Pre-trained Transformer 3) model. GPT-3 is an advanced language processing AI model developed by OpenAI, capable of understanding and generating human-like text. Azure Open AI provides a platform for developers to build applications and solutions that leverage the power of GPT-3 for natural language understanding, conversation, translation, content generation, and more. It offers an API-based interface to integrate GPT-3 capabilities into various applications and services."
      }
    }
  ],
  "usage": {
    "completion_tokens": 118,
    "prompt_tokens": 13,
    "total_tokens": 13

## Reading the resume

In [9]:
resume_file = "resume.txt"
!ls $resume_file -lh

-rwxrwxrwx 1 root root 1.8K Sep  7 10:39 resume.txt


In [10]:
with open(resume_file, "r") as f:
    doc = f.readlines()

doc = " ".join(doc)

In [11]:
print(doc[:500])

John Water
 Phone: (123) 456 78 99
 Email: john.water@hello.com
 Website: www.johnwater.com
 Linkedin: https://www.linkedin.com/jwater
 Address: 25 rue des chantiers 75012 Paris
 Date of birth: 20-Jan-1986
 Place of Birth: Versailles
 Nationality: French
 
 
 Objective
 Data Scientist with PhD in Physics and 10+ years of industrial experience. Two years of working experience in Data Analysis team of LIGO Scientific Collaboration [$3M Special Breakthrough Prize winner of 2016]. Over ten years of 


## Asking questions

In [12]:
def resume_qna(document, prompt, model):
    """
    Analysing resume with Azure Open AI
    """
    content = f""" {document}
      \n###
      \n{prompt}
    """

    messages = [{"role": "user", "content": content}]
    
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop="###",
    )

    return response

In [13]:
prompt = "Extract the key sections from the resume above into json."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 {
  "name": "John Water",
  "phone": "(123) 456 78 99",
  "email": "john.water@hello.com",
  "website": "www.johnwater.com",
  "linkedin": "https://www.linkedin.com/jwater",
  "address": "25 rue des chantiers 75012 Paris",
  "date_of_birth": "20-Jan-1986",
  "place_of_birth": "Versailles",
  "nationality": "French",
  "objective": "Data Scientist with PhD in Physics and 10+ years of industrial experience. Two years of working experience in Data Analysis team of LIGO Scientific Collaboration [$3M Special Breakthrough Prize winner of 2016]. Over ten years of successful research experience in both theoretical and datascience. Strong problem-solving and analytical skills. Advanced programming proficiency. Certified in Data Analysis and Machine Learning on Azure and AWS services.",
  "skills": "Azure, Azure AI, Azure ML, Data Mining, Data Analysis, Machine Learning, Python, R, MATLAB, Sphinx, Mathematica, Maple, GIT, Azure Custom Vision, Azure Open AI, SPSS, time series.",
  "work

In [14]:
prompt = "Can you describe this resume in 3 lines?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 This resume belongs to John Water, a data scientist with a PhD in Physics and 10+ years of industrial experience. He has worked on the Data Analysis team of the LIGO Scientific Collaboration and has strong problem-solving and analytical skills. He is proficient in programming languages such as Python and R, and is certified in Data Analysis and Machine Learning on Azure and AWS services.


In [15]:
prompt = "Est-ce que le candidat réside en France? Si oui dans quelle ville?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Oui, le candidat réside à Paris, en France.


In [16]:
prompt = "What are the 10 keywords of this resume?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 1. Data Scientist
2. PhD in Physics
3. Data Analysis
4. LIGO Scientific Collaboration
5. Problem-solving
6. Analytical skills
7. Programming proficiency
8. Azure
9. Machine Learning
10. Python


In [17]:
prompt = "What the location of this candidate?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 The location of this candidate is Paris, France.


In [18]:
prompt = "What is the place of birth? Is it in France?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Yes, the place of birth is Versailles, which is in France.


In [19]:
prompt = "What is the distance from the location and the place of birth from this candidate?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 The distance from the location (25 rue des chantiers 75012 Paris) to the place of birth (Versailles) is approximately 17 kilometers.


In [20]:
prompt = "How many companies has this candidate worked for? Give the names."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 This candidate has worked for three companies. The names of the companies are:

1. ABCD Corporation
2. XYZ Corporation
3. The PyCompany


In [21]:
prompt = "What are the people informations (name, address, email, linkedin profile) and display it in a json?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 {
  "name": "John Water",
  "address": "25 rue des chantiers 75012 Paris",
  "email": "john.water@hello.com",
  "linkedin": "https://www.linkedin.com/jwater"
}


In [22]:
prompt = "Do we have some computer vision skills in this resume? Explain."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Yes, there are computer vision skills mentioned in this resume. The candidate states that they have "Deep Learning expertise mainly in computer vision with PyTorch" in their role as a Python programmer at The PyCompany. This indicates that they have experience and knowledge in computer vision techniques and have worked with PyTorch, a popular deep learning framework often used for computer vision tasks.


In [23]:
prompt = "Do we have some Azure ML skills in this resume? Just answer by Yes or No."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Yes


In [24]:
prompt = "How many years of professional experience per company do we have in this resume? Display the dates."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 - ABCD Corporation: 3 years (Jan 2020 - Sept 2023)
- XYZ Corporation: 4 years (Sept 2015 - Sept 2019)
- The PyCompany: 7 years (Jan 2008 - August 2015)


In [25]:
prompt = "List all the skills mentionned by alphabetical order."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Azure, Azure AI, Azure Custom Vision, Azure ML, Azure Open AI, Data Analysis, Data Mining, GIT, Machine Learning, Maple, MATLAB, Python, R, Sphinx, SPSS, time series


In [26]:
prompt = "Do we have some Oracle experience? Just say Yes or No."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 No.


In [27]:
prompt = "Do we have some Azure experience? Explain."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 Yes, the individual has Azure experience. They have listed Azure, Azure AI, Azure ML, Azure Custom Vision, and Azure Open AI as skills. They are also certified in Data Analysis and Machine Learning on Azure and AWS services.


In [28]:
prompt = "List the Microsoft skills mentionned and display in a json structure."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 {
  "Azure": true,
  "Azure AI": true,
  "Azure ML": true,
  "Azure Custom Vision": true,
  "Azure Open AI": true
}
